# LangChain 101: Ruby on Rails for Generative AI

LangChain is known as an easy way to produce chatbots, but is actually much more: an onramp to generative AI. LangChain is like [Ruby on Rails](https://en.wikipedia.org/wiki/Ruby_on_Rails) [which powered Web 2.0] for generative artificial intelligence.

## LangChain 101: Retrieval Augmented Generation

I am obsessed with _network motifs_ - statistically significant patterns called _graphlets_ that appear in _complex networks_.

<br />
<center>
    <img src="../images/5-graphlets.png" width="600px" />
</center>
<br />
<center>
    <a href="https://www.nature.com/articles/s41598-020-69795-1">Exploiting graphlet decomposition to explain the structure of complex networks: the GHuST framework</a>, Espejo et al., 2020.
</center>

